# Summary

`Try getting GPT Neo predictions using the Huggingface API.`

In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import requests

from jabberwocky.config import C
from jabberwocky.openai_utils import load_prompt, load_openai_api_key
from jabberwocky.utils import load_huggingface_api_key
from htools import *

In [4]:
cd_root()

Current directory: /Users/hmamin/jabberwocky


In [24]:
HF_API_KEY = load_huggingface_api_key()
HEADERS = {'Authorization': f'Bearer api_{HF_API_KEY}'}
URL_FMT = 'https://api-inference.huggingface.co/models/{}'
# These accept different parameters. For now just start with the basics, but
# keep these around in case I want to do something with them later.
_task2suff = {'generate': 'EleutherAI/gpt-neo-2.7B',
              'summarize': 'facebook/bart-large-cnn',
              'chat': 'microsoft/DialoGPT-large',
              'q&a': 'deepset/roberta-base-squad2'}
TASK2URL = DotDict({k: URL_FMT.format(v) for k, v in _task2suff.items()})
NEO_URL = URL_FMT.format(_task2suff['generate'])

In [40]:
def query_gpt_neo(prompt, top_k=None, top_p=None, temperature=1.0, 
                  repetition_penalty=None, max_tokens=256, api_key=None,
                  **kwargs):
    headers = {'Authorization': 
               f'Bearer api_{api_key or load_huggingface_api_key()}'}
    # Notice the names don't always align with parameter names - I wanted 
    # those to be more consistent with query_gpt3() function.
    data = {'input': prompt, 'top_k': top_k, 'top_p': top_p, 
            'temperature': temperature, 'max_new_tokens': max_tokens}
    r = requests.post(NEO_URL, headers=headers, data=json.dumps(data))
    return r.json()
#     return r.json()[0]['generated_text']

In [41]:
text = 'Why does the'
res = query_gpt_neo(text)

print(res)

[[{'generated_text': 'input>\n\nThe code of the form is the same as the one used for the other forms in the page:\n<form id="form2" action="index.php" method="post">\n    <input type="'}], [{'generated_text': 'top_k, k=0 ... N) of size N x 4\n\nThe values within the matrix are as follows:\n0 = value_0\n1 = value_1\n2 = value_2\n3 = value_3'}], [{'generated_text': 'top_pwm1_cntl),\n\t(0x0f110000 + 0x20 * i + 0x9b * 2) = PM_CTRL1_PWM_OFF_FAST,\n\t'}], [{'generated_text': 'temperature (5-40°C), and humidity.\n\nProtein recovery was calculated as the remaining protein percentage per total soluble protein content after precipitation. Protein recovery percentage was calculated as follows:\n\nProtein recovery (%) = (Protein'}], [{'generated_text': 'max_new_tokens=50,token_limit=10**\n\nNote that this is just one of many ways you can change your token limit. This is just an example.\nHere are some more options:\n\nSet'}]]


In [48]:
for row in res:
    print(row[0]['generated_text'])

input>

The code of the form is the same as the one used for the other forms in the page:
<form id="form2" action="index.php" method="post">
    <input type="
top_k, k=0 ... N) of size N x 4

The values within the matrix are as follows:
0 = value_0
1 = value_1
2 = value_2
3 = value_3
top_pwm1_cntl),
	(0x0f110000 + 0x20 * i + 0x9b * 2) = PM_CTRL1_PWM_OFF_FAST,
	
temperature (5-40°C), and humidity.

Protein recovery was calculated as the remaining protein percentage per total soluble protein content after precipitation. Protein recovery percentage was calculated as follows:

Protein recovery (%) = (Protein
max_new_tokens=50,token_limit=10**

Note that this is just one of many ways you can change your token limit. This is just an example.
Here are some more options:

Set


In [33]:
dates_kwargs = load_prompt('short_dates', verbose=False)
res = query_gpt_neo(**dates_kwargs)

print(res)

Why does the world go so crazy so quickly???

If you are a person who has experienced any time of loss, pain or sadness you should know just how overwhelming life can be during that time.

The best way to get through an


In [37]:
res = query_gpt_neo('What makes fish so')

print(res)

Why does the world go so crazy so quickly???

If you are a person who has experienced any time of loss, pain or sadness you should know just how overwhelming life can be during that time.

The best way to get through an
